<a href="https://colab.research.google.com/github/AdamWatson91/fb_marketplace/blob/main/product_classification_mixed_modal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [103]:
!mkdir data
!cp "/content/drive/MyDrive/text_model_data.npy" "data/text_model_data.npy"

mkdir: cannot create directory ‘data’: File exists


In [104]:
# Install module for Bert model
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Module imports and dataset imports

In [105]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torchvision
import torch.nn.functional as F
import numpy as np
from PIL import Image
from torchvision.transforms import transforms
from torch.utils.data import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import time
import copy
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.nn.functional as F
from google.colab import drive
from transformers import BertTokenizer
from transformers import BertModel

# Load the TensorBoard notebook extension
# %reload_ext tensorboard
# %tensorboard --logdir runs

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

products = np.load("data/text_model_data.npy", allow_pickle=True)
products = pd.DataFrame(products, columns=['product_id', 'product_description', 'category', 'img_id', 'img_array'])
products

Using cuda for inference


UnpicklingError: ignored

Create text dataset made up of tokenised text and numerical labels

In [ ]:
class ImageTextDataset(Dataset):
    def __init__(self, data, text_pad_length=None, transform=None):
        super().__init__()
        self.labels = data['category'].to_list()
        self.num_classes = len(set(self.labels))
        self.descriptions = data['product_description']
        self.descriptions_list = self.descriptions.to_list()
        # self.max_length_description = max(self.descriptions, key=len)
        if text_pad_length is None:
          avg_description_length = sum( map(len, self.descriptions_list) ) / len(self.descriptions_list)
          self.pad_length = round(avg_description_length)
        else:
          self.pad_length = text_pad_length
        self.img_array = data['img_array']
        self.files = data['img_id']
        # NEED TO SAVE THESE
        self.category_encoder = {y: x for (x, y) in enumerate(set(self.labels))}
        self.category_decoder = {x: y for (x, y) in enumerate(set(self.labels))}
        # self.vocab = self.create_vocab(self.descriptions_list)
        # self.vocab_size = len(self.vocab)
        self.tokeniser = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
        self.transform = transform
        if transform is None:
          self.transform = transforms.Compose([
                                               transforms.RandomHorizontalFlip(p=0.3),
                                               transforms.ToTensor(),
                                               transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
          ])
                                               
        assert len(self.descriptions_list) == len(self.labels)

    def __getitem__(self, index):
        label = self.category_encoder[self.labels[index]]
        label = torch.as_tensor(label).to(device)
        image = Image.fromarray(self.img_array[index])
        image = self.transform(image).to(device)
        sentence = self.descriptions.iloc[index]
        encoded = self.tokeniser.batch_encode_plus([sentence], max_length=self.pad_length, padding='max_length', truncation=True)
        encoded = {key: torch.LongTensor(value) for key, value in encoded.items()}
        with torch.no_grad():
          description = self.model(**encoded).last_hidden_state.swapaxes(1,2)

        description = description.squeeze(0).to(device)

        return (image, description, label)

    def __len__(self):
        return len(self.descriptions_list)



Can you do data augmentation with NLP?


In [ ]:
class BertCNN(torch.nn.Module):
    def __init__(self, embedding_size=768, out_size=13):
        super(BertCNN, self).__init__()
        self.layers = torch.nn.Sequential(
            nn.Conv1d(embedding_size, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(256, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(64, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            # nn.Dropout(),
            nn.Flatten(),
            nn.Linear(141 , 256),
            nn.ReLU(),
            nn.Linear(141, out_size)
            )

    def forward(self, X):
        X = self.layers(X)
        return X



In [ ]:
class CombinedModel(nn.Module):
  def __init__(self, embedding_size=768, out_size=13):
    super(CombinedModel, self).__init__()
    resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
    out_features = resnet50.fc.out_features
    self.image_classifier = nn.Sequential(resnet50, nn.Linear(out_features, 128)).to(device)
    self.text_classifier = BertCNN()
    self.main = nn.Sequential(nn.Linear(256, out_size))

  def forward(self, image_features, text_features):
    image_features = self.image_classifier(image_features)
    text_features = self.text_classifier(text_features)
    combined_features = torch.cat((image_features, text_features), 1)
    combined_features = self.main(combined_features)
    return combined_features

In [ ]:
dataset = ImageTextDataset(products, text_pad_length=292, transform=None)
batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
combined = CombinedModel()
combined.to(device)
criterion = nn.CrossEntropyLoss()
optimiser = optim.Adam(combined.parameters(), lr = 0.001)
epochs = 5


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [ ]:
from tqdm import tqdm

for epoch in range(epochs):
    since = time.time()
    hist_accuracy = []
    hist_loss = []
    accuracy = 0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, batch in pbar:
        image_features, text_features, labels = batch
        image_features = image_features.to(device)
        text_features = text_features.to(device)
        labels = labels.to(device)
        optimiser.zero_grad()
        outputs = combined(image_features, text_features)
        loss = criterion(outputs, labels)
        loss.backward()
        accuracy = torch.mean((torch.argmax(outputs, dim=1) == labels).float().item())
        hist_accuracy.append(accuracy)
        hist_loss.append(loss.item())
        optimiser.step()
        pbar.set_description(f"Epoch = {epoch+1}/{epochs}. loss = {loss.item():.4f} Acc = {round(torch.sum(torch.argmax(outputs, dim=1) == labels).item()/len(labels), 2)}, Total_acc = {round(np.mean(hist_accuracy), 2)}" )

time_elapsed = time.time() - since
print(time_elapsed)

  0%|          | 0/364 [00:19<?, ?it/s]


RuntimeError: ignored

In [ ]:
def train_model(model, num_epochs, optimiser, criterion, scheduler=None, model_load_location=None):
    writer = SummaryWriter()
    
    since = time.time()

    # keep best model at every epoch by copying weights
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        # print current epoch
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        # pbar = tqdm(enumerate(['train', 'val']), total=len(['train', 'val']))
        for phase in ['train', 'val']:
            if phase == 'train':
                print("Training...")
                model.train()  # Set model to training mode
            else:
                print("Validating...")
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for batch_idx, batch in enumerate(data_loader[phase]):
                print(batch_idx)
                features, labels = batch
                features, labels = features.to(device), labels.to(device)

                # zero the parameter gradients
                optimiser.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(features)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # loss = F.cross_entropy(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimiser.step()

                # statistics
                running_loss += loss.item() * features.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train' and scheduler != None:
                scheduler.step()
            print(dataset_sizes[phase])
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            writer.add_scalar('Epoch Loss', epoch_loss, epoch)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == "train":
                writer.add_scalar("Training Accuracy", epoch_acc*100, epoch)
            else:
                writer.add_scalar("Validation Accuracy", epoch_acc*100, epoch)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

            writer.flush()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)

    # if model_save_location != None:
    #     torch.save(model.state_dict(), model_save_location)

    return model




In [ ]:
def train_validation_test_split(data, train_percent, val_percent, batch_size, shuffle=True):
    data_size = len(data)
    train_size = int(train_percent * data_size)
    val_size = int(val_percent * data_size)
    test_size = data_size - (train_size + val_size)

    indices = list(range(data_size))
    if shuffle:
      np.random.shuffle(indices)

    train_split_indice = indices[:train_size]
    val_split_indice = indices[train_size:(val_size + train_size)]
    test_split_indice =indices[(val_size + train_size):data_size]

    # Create dictionary of dataset sizes
    dataset_sizes = {"train": train_size,
    "val": val_size,
    "test": test_size
    }

    splits = {"train": train_split_indice,
    "val": val_split_indice,
    "test": test_split_indice
    }

    samplers = {"train": SubsetRandomSampler(splits["train"]),
        "val": SubsetRandomSampler(splits["val"]),
        "test": SubsetRandomSampler(splits["test"])
        }
    
    dataset = {phase: data
        for phase in ["train", "val", "test"]}

    data_loader = {phase: DataLoader(dataset[phase], batch_size=batch_size, sampler=samplers[phase])
        for phase in ["train", "val", "test"]}

    return data_loader, dataset_sizes


{'train': 4570, 'val': 1305, 'test': 654}
tensor([[[-0.5891,  0.3420, -0.3100,  ..., -0.2536, -0.1784, -0.3610],
         [-0.1581, -0.3863, -0.6790,  ..., -0.6691, -0.6779, -0.8079],
         [ 0.7513,  1.1665,  0.5932,  ...,  0.5469,  0.7161,  0.5418],
         ...,
         [-0.3239,  0.4387, -0.0340,  ...,  0.2407,  0.3058,  0.3273],
         [ 0.1625,  0.6132,  0.0532,  ...,  0.3102,  0.3489,  0.3322],
         [ 0.2730, -0.3002,  0.1331,  ..., -0.4400, -0.3763, -0.4696]],

        [[-0.9658,  0.0211, -0.2694,  ...,  0.0244, -0.2702, -0.2160],
         [-0.0404,  0.3615,  0.4668,  ..., -0.0359, -0.3199, -0.2416],
         [ 0.3220,  0.4451,  0.4582,  ...,  0.3325,  0.3348,  0.4227],
         ...,
         [ 0.0284,  0.2936,  0.1670,  ..., -0.1714, -0.0246,  0.0571],
         [-0.0577,  1.0571,  0.1832,  ..., -0.1434, -0.3309, -0.1816],
         [ 0.2646,  0.0517, -1.8421,  ..., -0.2691, -0.3191, -0.4349]],

        [[-0.8197,  0.4421,  0.1649,  ..., -0.0096, -0.1730, -0.1272],
   